# **Tarea 5: Algoritmos de Clustering**
**Nombre:** José Luis Cádiz

**ReadMe:** Ejecutar todo el código, si se ejecuta solo una parte aislada pueden existir errores producto de que hay variables que se sobreescriben. 

# **Introducción:**

**En el contexto del curso ''Inteligencia Computacional'', se presentará a continuación el desarrollo de la ''Tarea 5'' del curso. Esta tarea pretende clusterizar un conjunto de datos  que contiene 22 caracteristicas de 10 ranas distintas utilizando tres algoritmos de clustering: k-means, DBSCAN y clustering aglomerativo, esto con el objetivo de analizar el desempeño de cada clusterización según diversas metricas.**

**En esta experiencia se usarán las librerías pandas, numpy y scikit-learn para manipular, procesar y clusterizar los datos.**

**A continuación se explica a grandes razgos lo que se realizará en la tarea: En la parte 1, se implementará un código que permita leer la base de datos, separando las caractericas y labels, transformando estos ultimos en números de 0 a 9.**

**En la parte 2, se modificará la función bench_k_means( ) agregando una columna extra, correspondiente al número de
clusters.** 

**En la parte 3, se implementará una nueva función bench_clustering2( ), tomando como base la función
bench_k_means( ) modificada en el punto anterior. Esta función será usada para obtener un
benchmark para k-means, DBSCAN y clustering aglomerativo.**

**En la parte 4, se harán pruebas con distintas variantes de los algoritmos mencionados.**

**En la parte 5, se analizarán los resultados obtenidos comparando el desempeño de cada algoritmo según las metricas obtenidas, con las variantes de considerar los outliers como un cluster extra  y el efecto del PCA.**


**Finalmente, se termina el documento con las conclusiones principales de esta experiencia.**

In [ ]:
#Importando librerias:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

#Libreria sklearn:
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA

#Para el calculo de tiempos de entrenamiento:
from time import time

#Importar datos desde google drive:
from google.colab import drive  #Se uso para no subir el archivo cada vez que se avanzaba en la tarea
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Parte 1:** Implemente un código que lea la base de datos. Se recomienda usar la biblioteca Pandas. Los datos
se deben dividir en características (MFCCs_1 – MFCCs_22) y labels (Species). Los labels deben
transformarse a números en el rango 0-9:

In [ ]:
# Lectura de la Base de Datos:
path = 'drive/My Drive/Inteligencia Computacional/Frogs_MFCCs.csv' # Cambiar path*
datos=pd.read_csv(path)
print(datos.head())
print(datos.shape)


   MFCCs_ 1  MFCCs_ 2  MFCCs_ 3  ...      Genus         Species  RecordID
0       1.0  0.152936 -0.105586  ...  Adenomera  AdenomeraAndre         1
1       1.0  0.171534 -0.098975  ...  Adenomera  AdenomeraAndre         1
2       1.0  0.152317 -0.082973  ...  Adenomera  AdenomeraAndre         1
3       1.0  0.224392  0.118985  ...  Adenomera  AdenomeraAndre         1
4       1.0  0.087817 -0.068345  ...  Adenomera  AdenomeraAndre         1

[5 rows x 26 columns]
(7195, 26)


In [ ]:
# Características:
features=datos.iloc[:,0:22]
data=features
data.head()

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,MFCCs_11,MFCCs_12,MFCCs_13,MFCCs_14,MFCCs_15,MFCCs_16,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,0.188654,-0.075622,-0.156436,0.082245,0.135752,-0.024017,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,0.270958,-0.095004,-0.254341,0.022786,0.163320,0.012022,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,0.266064,-0.072827,-0.237384,0.050791,0.207338,0.083536,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,0.267279,-0.162258,-0.317084,-0.011567,0.100413,-0.050224,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,0.332695,-0.100749,-0.298524,0.037439,0.219153,0.062837,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244


In [ ]:
# Labels:
labels=datos.iloc[:,24]
labels

0       AdenomeraAndre
1       AdenomeraAndre
2       AdenomeraAndre
3       AdenomeraAndre
4       AdenomeraAndre
             ...      
7190       ScinaxRuber
7191       ScinaxRuber
7192       ScinaxRuber
7193       ScinaxRuber
7194       ScinaxRuber
Name: Species, Length: 7195, dtype: object

In [ ]:
# Se extraen las distintas clases:
names=labels.unique().tolist()
print(names)
len(names)

['AdenomeraAndre', 'Ameeregatrivittata', 'AdenomeraHylaedactylus', 'HylaMinuta', 'HypsiboasCinerascens', 'HypsiboasCordobae', 'LeptodactylusFuscus', 'OsteocephalusOophagus', 'Rhinellagranulosa', 'ScinaxRuber']


10

In [ ]:
# Transformación de Labels:
Class=0
for i in names:
  labels=labels.replace(i,Class)
  Class=Class+1
print(labels.unique().tolist())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


**Parte 2:** Modifique la función bench_k_means( ) agregando una columna extra, correspondiente al número de clusters:

In [ ]:
# Función predefinida
def bench_k_means(kmeans, name, data, labels):

    t0 = time()
    estimator = make_pipeline(StandardScaler(), kmeans).fit(data)
    fit_time = time() - t0
    results = [name, fit_time, estimator[-1].inertia_]

    # Define the metrics which require only the true labels and estimator
    # labels
    clustering_metrics = [
        metrics.homogeneity_score,
        metrics.completeness_score,
        metrics.v_measure_score,
        metrics.adjusted_rand_score,
        metrics.adjusted_mutual_info_score,
    ]
    results += [m(labels, estimator[-1].labels_) for m in clustering_metrics]

    # The silhouette score requires the full dataset
    results += [
        metrics.silhouette_score(data, estimator[-1].labels_,
                                 metric="euclidean", sample_size=300,)
    ]

    # Número de Clusters:
    x=pd.DataFrame(estimator[-1].labels_)
    x.set_axis(['cluster'], axis=1, inplace=True)
    cluster_group=x['cluster'].unique().tolist() 
    num=len(cluster_group)
    results+=[num]

    # Show the results
    formatter_result = ("{:9s}\t{:.3f}s\t{:.0f}\t{:.3f}\t{:.3f}"
                        "\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.0f}")
    print(formatter_result.format(*results))

In [ ]:
print(90 * '_')
print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsil\t# Clusters')

kmeans = KMeans(init="k-means++", n_clusters=10, n_init=4,
                random_state=0)
bench_k_means(kmeans=kmeans, name="k-means++", data=data, labels=labels)

kmeans = KMeans(init="random", n_clusters=6, n_init=4, random_state=0)
bench_k_means(kmeans=kmeans, name="random", data=data, labels=labels)

pca = PCA(n_components=9).fit(data)
kmeans = KMeans(init=pca.components_, n_clusters=9, n_init=1)
bench_k_means(kmeans=kmeans, name="PCA-based", data=data, labels=labels)

print(90 * '_')

__________________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	sil	# Clusters
k-means++	0.274s	51996	0.708	0.571	0.633	0.432	0.631	0.254	10
random   	0.115s	67625	0.588	0.614	0.600	0.499	0.600	0.255	6
PCA-based	0.109s	62881	0.719	0.718	0.719	0.871	0.718	0.361	9
__________________________________________________________________________________________


**Parte 3:** Implemente una nueva función bench_clustering2( ), tomando como base la función
bench_k_means( ) modificada en el punto anterior. Esta función será usada para obtener un
benchmark tanto de DBSCAN como del clustering aglomerativo:

In [ ]:
def bench_clustering2(clustering, name, data, labels,opcion=True): 
    t0 = time()
    estimator = make_pipeline(StandardScaler(), clustering).fit(data)
    fit_time = time() - t0
    data1=data # Se redefine data para evitar algún eventual error al ser modificada
    labels1=labels # Se redefine labels para evitar algún eventual error al ser modificada

    if name=='k-means random' or name=='k-means++' : # Solo si el nombre es kmeans se calculará la metrica inertia_
      results = [name, fit_time, int(round(estimator[-1].inertia_,0))]
    else:
      results = [name, fit_time,None] # En otro caso, se obtiene None
    
    if opcion==True: # True no considera outlier como grupo
      u=pd.DataFrame(estimator[-1].labels_) # DataFrame de las clasificaciones para eliminar las filas correspondientes con la función drop
      x=estimator[-1].labels_.tolist() # lista de los clusters para la extracción de indices 
      indices=[ i  for i in range(len(x)) if x[i]==-1 ] # Extracción de indices en donde el cluster fue -1
  
      for i in indices: # Se itera sobre los indices donde se encontraron -1 para eliminar las filas asociadas a ese cluster
        data1=data1.drop(i,axis=0) # Se eliminan filas asociadas a los clusters -1
        labels1=labels1.drop(i,axis=0) # Se eliminan labels asociados a los clusters -1
        u=u.drop(i,axis=0) # Se eliminan -1 de los clusters
      u=u.to_numpy().flatten() # Correción de un problema de dimensionalidad

      # Metricas:
      clustering_metrics = [
        metrics.homogeneity_score,
        metrics.completeness_score,
        metrics.v_measure_score,
        metrics.adjusted_rand_score,
        metrics.adjusted_mutual_info_score,]
      results += [m(labels1, u)  for m in clustering_metrics] # Metricas reciben como input los labels y clusters actualizados sin los clusters asociados a -1
      
      try: # Se intenta calcular la metrica silhouette
        results += [
        round(metrics.silhouette_score(data1, u,
                                  metric="euclidean"),3)] # Metricas reciben como input los labels y clusters actualizados sin los clusters asociados a -1
      except: 
        results +=[None] # En el caso que no pueda ser calculada se obtiene None

      # Se contabilizan los Clusters:
      x=pd.DataFrame(estimator[-1].labels_)
      x.set_axis(['cluster'], axis=1, inplace=True)
      cluster_group=x['cluster'].unique().tolist()
      if -1 in cluster_group: # Si hay cluster -1 este se elimina
        cluster_group.remove(-1)
      num=len(cluster_group)
      results+=[num]
      
     
    if opcion==False:  # False considera outlier como grupo
      
      # Metricas:
      clustering_metrics = [
        metrics.homogeneity_score,
        metrics.completeness_score,
        metrics.v_measure_score,
        metrics.adjusted_rand_score,
        metrics.adjusted_mutual_info_score,]
      results += [m(labels, estimator[-1].labels_)  for m in clustering_metrics]

      try: # Se intenta calcular la metrica silhouette
        results += [
          round(metrics.silhouette_score(data, estimator[-1].labels_,
                                  metric="euclidean"),3)]
      except:
        results +=[None] # En el caso que no pueda ser calculada se obtiene None

      # Se contabilizan los Clusters:
      x=pd.DataFrame(estimator[-1].labels_)
      x.set_axis(['cluster'], axis=1, inplace=True)
      cluster_group=x['cluster'].unique().tolist() # Se consideran clusters -1
      num=len(cluster_group)
      results+=[num]

    # Show the results
    formatter_result = ("{:9s}\t{:.3f}s\t{:}\t{:.3f}\t{:.3f}"
                        "\t{:.3f}\t{:.3f}\t{:.3f}\t{:}\t{:.0f}")
    print(formatter_result.format(*results))

**Parte 4:** Hacer pruebas con distintas variantes de algoritmos:

**a)** K-Means (con inicialización al azar):

In [ ]:
print(90 * '_')
print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsil\t# Clusters')

kmeans = KMeans(init="random", n_clusters=10, n_init=4, random_state=0)
bench_clustering2(clustering=kmeans, name="k-means random", data=data, labels=labels)

print(90 * '_')

__________________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	sil	# Clusters
k-means random	0.311s	54400	0.641	0.507	0.566	0.326	0.565	0.224	10
__________________________________________________________________________________________


**b)** K-means++:

In [ ]:
print(90 * '_')
print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsil\t# Clusters')

kmeans = KMeans(init="k-means++", n_clusters=10, n_init=4,
                random_state=0)
bench_clustering2(clustering=kmeans, name="k-means++", data=data, labels=labels)

print(90 * '_')

__________________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	sil	# Clusters
k-means++	0.297s	51996	0.708	0.571	0.633	0.432	0.631	0.236	10
__________________________________________________________________________________________


**c)** DBSCAN con épsilon por defecto:

In [ ]:
print(90 * '_')
print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsil\t# Clusters')
clustering = DBSCAN()
bench_clustering2(clustering=clustering, name="DBSCAN", data=data, labels=labels)
print(90 * '_')
x=pd.DataFrame(clustering.labels_)
x.set_axis(['cluster'], axis=1, inplace=True)
cluster_group=x['cluster'].unique().tolist()
print(cluster_group)

__________________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	sil	# Clusters
DBSCAN   	0.565s	None	1.000	0.327	0.493	0.256	0.483	0.246	8
__________________________________________________________________________________________
[-1, 0, 1, 2, 3, 4, 5, 6, 7]


**d)** DBSCAN con épsilon 0.7:

In [ ]:
print(90 * '_')
print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsil\t# Clusters')
clustering = DBSCAN(eps=0.7)
bench_clustering2(clustering=clustering, name="DBSCAN", data=data, labels=labels)
print(90 * '_')
x=pd.DataFrame(clustering.labels_)
x.set_axis(['cluster'], axis=1, inplace=True)
cluster_group=x['cluster'].unique().tolist()
print(cluster_group)

__________________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	sil	# Clusters
DBSCAN   	0.744s	None	1.000	0.585	0.738	0.450	0.730	0.546	13
__________________________________________________________________________________________
[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


**e)** DBSCAN con épsilon 0.2:

In [ ]:
print(90 * '_')
print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsil\t# Clusters')
clustering = DBSCAN(eps=0.2)
bench_clustering2(clustering=clustering, name="DBSCAN", data=data, labels=labels)
print(90 * '_')
x=pd.DataFrame(clustering.labels_)
x.set_axis(['cluster'], axis=1, inplace=True)
cluster_group=x['cluster'].unique().tolist()
print(cluster_group)

__________________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	sil	# Clusters
DBSCAN   	0.395s	None	1.000	0.000	0.000	0.000	0.000	0.52	2
__________________________________________________________________________________________
[-1, 0, 1]


**f)** DBSCAN con épsilon por defecto, agregando outliers a cluster extra:

In [ ]:
print(90 * '_')
print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsil\t# Clusters')
clustering = DBSCAN( )
bench_clustering2(clustering=clustering, name="DBSCAN", data=data, labels=labels,opcion=False)
print(90 * '_')
x=pd.DataFrame(clustering.labels_)
x.set_axis(['cluster'], axis=1, inplace=True)
cluster_group=x['cluster'].unique().tolist()
print(cluster_group)

__________________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	sil	# Clusters
DBSCAN   	0.604s	None	0.074	0.451	0.127	0.049	0.123	-0.21	9
__________________________________________________________________________________________
[-1, 0, 1, 2, 3, 4, 5, 6, 7]


**g)** DBSCAN con épsilon 0.7, agregando outliers a cluster extra:

In [ ]:
print(90 * '_')
print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsil\t# Clusters')
clustering = DBSCAN(eps=0.7)
bench_clustering2(clustering=clustering, name="DBSCAN", data=data, labels=labels,opcion=False)
print(90 * '_')
x=pd.DataFrame(clustering.labels_)
x.set_axis(['cluster'], axis=1, inplace=True)
cluster_group=x['cluster'].unique().tolist()
print(cluster_group)

__________________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	sil	# Clusters
DBSCAN   	0.710s	None	0.179	0.545	0.270	0.133	0.265	-0.343	14
__________________________________________________________________________________________
[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


**h)** DBSCAN con épsilon 0.2, agregando outliers a cluster extra:


In [ ]:
print(90 * '_')
print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsil\t# Clusters')
clustering = DBSCAN(eps=0.2)
bench_clustering2(clustering=clustering, name="DBSCAN", data=data, labels=labels,opcion=False)
print(90 * '_')
x=pd.DataFrame(clustering.labels_)
x.set_axis(['cluster'], axis=1, inplace=True)
cluster_group=x['cluster'].unique().tolist()
print(cluster_group)

__________________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	sil	# Clusters
DBSCAN   	0.345s	None	0.011	0.677	0.022	0.006	0.021	-0.094	3
__________________________________________________________________________________________
[-1, 0, 1]


**i)** Clustering aglomerativo:

In [ ]:
print(90 * '_')
print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsil\t# Clusters')
clustering = AgglomerativeClustering(n_clusters=10)
bench_clustering2(clustering=clustering, name="Agglomerative", data=data, labels=labels)
print(90 * '_')
x=pd.DataFrame(clustering.labels_)
x.set_axis(['cluster'], axis=1, inplace=True)
cluster_group=x['cluster'].unique().tolist()
print(cluster_group)

__________________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	sil	# Clusters
Agglomerative	1.824s	None	0.789	0.661	0.719	0.590	0.718	0.239	10
__________________________________________________________________________________________
[4, 0, 1, 3, 8, 9, 6, 2, 5, 7]


**j)** Repetir todas las pruebas anteriores, después de usar PCA sobre los datos para reducirlos a
2 dimensiones:

In [ ]:
# PCA a data:
pca=PCA(n_components=2)
pca.fit(data)
data=pd.DataFrame(pca.transform(data))
print(data.head())

          0         1
0 -0.469179 -0.695615
1 -0.514870 -0.721527
2 -0.476569 -0.704513
3 -0.537135 -0.590022
4 -0.501557 -0.768868


**a)** K-Means (con inicialización al azar):

In [ ]:
print(90 * '_')
print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsil\t# Clusters')

kmeans = KMeans(init="random", n_clusters=10, n_init=4, random_state=0)
bench_clustering2(clustering=kmeans, name="k-means random", data=data, labels=labels)

print(90 * '_')

__________________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	sil	# Clusters
k-means random	0.206s	1044	0.616	0.479	0.539	0.318	0.537	0.398	10
__________________________________________________________________________________________


**b)** K-means++:

In [ ]:
print(90 * '_')
print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsil\t# Clusters')

kmeans = KMeans(init="k-means++", n_clusters=10, n_init=4,
                random_state=0)
bench_clustering2(clustering=kmeans, name="k-means++", data=data, labels=labels)

print(90 * '_')

__________________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	sil	# Clusters
k-means++	0.162s	943	0.654	0.497	0.565	0.355	0.564	0.413	10
__________________________________________________________________________________________


**c)** DBSCAN con épsilon por defecto:

In [ ]:
print(90 * '_')

print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsil\t# Clusters')
clustering = DBSCAN()
bench_clustering2(clustering=clustering, name="DBSCAN", data=data, labels=labels)

print(90 * '_')

x=pd.DataFrame(clustering.labels_)
x.set_axis(['cluster'], axis=1, inplace=True)
cluster_group=x['cluster'].unique().tolist()
print(cluster_group)

__________________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	sil	# Clusters
DBSCAN   	0.210s	None	0.000	1.000	0.000	0.000	0.000	None	1
__________________________________________________________________________________________
[0, -1]


**d)** DBSCAN con épsilon 0.7:

In [ ]:
print(90 * '_')

print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsil\t# Clusters')
clustering = DBSCAN(eps=0.7)
bench_clustering2(clustering=clustering, name="DBSCAN", data=data, labels=labels)

print(90 * '_')

x=pd.DataFrame(clustering.labels_)
x.set_axis(['cluster'], axis=1, inplace=True)
cluster_group=x['cluster'].unique().tolist()
print(cluster_group)

__________________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	sil	# Clusters
DBSCAN   	0.271s	None	-0.000	1.000	-0.000	0.000	-0.000	None	1
__________________________________________________________________________________________
[0, -1]


**e)** DBSCAN con épsilon 0.2:

In [ ]:
print(90 * '_')

print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsil\t# Clusters')
clustering = DBSCAN(eps=0.2)
bench_clustering2(clustering=clustering, name="DBSCAN", data=data, labels=labels)

print(90 * '_')

x=pd.DataFrame(clustering.labels_)
x.set_axis(['cluster'], axis=1, inplace=True)
cluster_group=x['cluster'].unique().tolist()
print(cluster_group)

__________________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	sil	# Clusters
DBSCAN   	0.204s	None	0.002	0.198	0.004	-0.001	0.003	0.031	3
__________________________________________________________________________________________
[0, -1, 1, 2]


**f)** DBSCAN con épsilon por defecto, agregando outliers a cluster extra:

In [ ]:
print(90 * '_')

print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsil\t# Clusters')
clustering = DBSCAN( )
bench_clustering2(clustering=clustering, name="DBSCAN", data=data, labels=labels,opcion=False)

print(90 * '_')

x=pd.DataFrame(clustering.labels_)
x.set_axis(['cluster'], axis=1, inplace=True)
cluster_group=x['cluster'].unique().tolist()
print(cluster_group)

__________________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	sil	# Clusters
DBSCAN   	0.212s	None	0.000	0.120	0.001	-0.000	0.000	0.484	2
__________________________________________________________________________________________
[0, -1]


**g)** DBSCAN con épsilon 0.7, agregando outliers a cluster extra:

In [ ]:
print(90 * '_')

print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsil\t# Clusters')
clustering = DBSCAN(eps=0.7)
bench_clustering2(clustering=clustering, name="DBSCAN", data=data, labels=labels,opcion=False)

print(90 * '_')

x=pd.DataFrame(clustering.labels_)
x.set_axis(['cluster'], axis=1, inplace=True)
cluster_group=x['cluster'].unique().tolist()
print(cluster_group)

__________________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	sil	# Clusters
DBSCAN   	0.268s	None	0.000	0.074	0.000	-0.000	-0.000	0.519	2
__________________________________________________________________________________________
[0, -1]


**h)** DBSCAN con épsilon 0.2, agregando outliers a cluster extra:


In [ ]:
print(90 * '_')

print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsil\t# Clusters')
clustering = DBSCAN(eps=0.2)
bench_clustering2(clustering=clustering, name="DBSCAN", data=data, labels=labels,opcion=False)

print(90 * '_')

x=pd.DataFrame(clustering.labels_)
x.set_axis(['cluster'], axis=1, inplace=True)
cluster_group=x['cluster'].unique().tolist()
print(cluster_group)

__________________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	sil	# Clusters
DBSCAN   	0.123s	None	0.004	0.103	0.007	0.001	0.005	0.019	4
__________________________________________________________________________________________
[0, -1, 1, 2]


**i)** Clustering aglomerativo:

In [ ]:
print(90 * '_')

print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsil\t# Clusters')
clustering = AgglomerativeClustering(n_clusters=10)
bench_clustering2(clustering=clustering, name="Agglomerative", data=data, labels=labels)

print(90 * '_')

x=pd.DataFrame(clustering.labels_)
x.set_axis(['cluster'], axis=1, inplace=True)
cluster_group=x['cluster'].unique().tolist()
print(cluster_group)

__________________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	sil	# Clusters
Agglomerative	1.333s	None	0.649	0.490	0.558	0.350	0.557	0.377	10
__________________________________________________________________________________________
[0, 4, 5, 3, 8, 2, 1, 9, 7, 6]


**Parte 5:** Analice los resultados obtenidos:

**a)** Compare los algoritmos aplicados en base a cuatro métricas (completeness, homogeneity, vmeasure,
silhouette), indicando: (i) cuál variante es mejor según cada métrica y (ii) cuál
variante es peor según cada métrica. Se debe considerar tanto los datos originales como los
casos en los cuales se aplicó PCA.

**A continuación se deja un cuadro resumen con los resultados para tener una mejor visualización.**

In [ ]:
print('Sin PCA')
print(90 * '_')
print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsil\t# Clusters')

data=features

kmeans = KMeans(init="random", n_clusters=10, n_init=4, random_state=0)
bench_clustering2(clustering=kmeans, name="k-means random", data=data, labels=labels)

kmeans = KMeans(init="k-means++", n_clusters=10, n_init=4,random_state=0)
bench_clustering2(clustering=kmeans, name="k-means++", data=data, labels=labels)

clustering = DBSCAN()
bench_clustering2(clustering=clustering, name="DBSCAN def", data=data, labels=labels)

clustering = DBSCAN(eps=0.7)
bench_clustering2(clustering=clustering, name="DBSCAN 0.7", data=data, labels=labels)

clustering = DBSCAN(eps=0.2)
bench_clustering2(clustering=clustering, name="DBSCAN 0.2", data=data, labels=labels)

clustering = DBSCAN( )
bench_clustering2(clustering=clustering, name="DBSCAN def-1", data=data, labels=labels,opcion=False)

clustering = DBSCAN(eps=0.7)
bench_clustering2(clustering=clustering, name="DBSCAN 0.7-1", data=data, labels=labels,opcion=False)

clustering = DBSCAN(eps=0.2)
bench_clustering2(clustering=clustering, name="DBSCAN 0.2-1", data=data, labels=labels,opcion=False)

clustering = AgglomerativeClustering(n_clusters=10)
bench_clustering2(clustering=clustering, name="Agglomerative", data=data, labels=labels)
print(90 * '_')

print('PCA')

print(90 * '_')

# PCA a data:
pca=PCA(n_components=2)
pca.fit(data)
data=pd.DataFrame(pca.transform(data))

kmeans = KMeans(init="random", n_clusters=10, n_init=4, random_state=0)
bench_clustering2(clustering=kmeans, name="k-means random", data=data, labels=labels)

kmeans = KMeans(init="k-means++", n_clusters=10, n_init=4,random_state=0)
bench_clustering2(clustering=kmeans, name="k-means++", data=data, labels=labels)

clustering = DBSCAN()
bench_clustering2(clustering=clustering, name="DBSCAN def", data=data, labels=labels)

clustering = DBSCAN(eps=0.7)
bench_clustering2(clustering=clustering, name="DBSCAN 0.7", data=data, labels=labels)

clustering = DBSCAN(eps=0.2)
bench_clustering2(clustering=clustering, name="DBSCAN 0.2", data=data, labels=labels)

clustering = DBSCAN( )
bench_clustering2(clustering=clustering, name="DBSCAN def-1", data=data, labels=labels,opcion=False)

clustering = DBSCAN(eps=0.7)
bench_clustering2(clustering=clustering, name="DBSCAN 0.7-1", data=data, labels=labels,opcion=False)

clustering = DBSCAN(eps=0.2)
bench_clustering2(clustering=clustering, name="DBSCAN 0.2-1", data=data, labels=labels,opcion=False)

clustering = AgglomerativeClustering(n_clusters=10)
bench_clustering2(clustering=clustering, name="Agglomerative", data=data, labels=labels)

print(90 * '_')

Sin PCA
__________________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	sil	# Clusters
k-means random	0.297s	54400	0.641	0.507	0.566	0.326	0.565	0.224	10
k-means++	0.257s	51996	0.708	0.571	0.633	0.432	0.631	0.236	10
DBSCAN def	0.556s	None	1.000	0.327	0.493	0.256	0.483	0.246	8
DBSCAN 0.7	0.744s	None	1.000	0.585	0.738	0.450	0.730	0.546	13
DBSCAN 0.2	0.385s	None	1.000	0.000	0.000	0.000	0.000	0.52	2
DBSCAN def-1	0.600s	None	0.074	0.451	0.127	0.049	0.123	-0.21	9
DBSCAN 0.7-1	0.690s	None	0.179	0.545	0.270	0.133	0.265	-0.343	14
DBSCAN 0.2-1	0.338s	None	0.011	0.677	0.022	0.006	0.021	-0.094	3
Agglomerative	1.829s	None	0.789	0.661	0.719	0.590	0.718	0.239	10
__________________________________________________________________________________________
PCA
__________________________________________________________________________________________
k-means random	0.243s	1044	0.616	0.479	0.539	0.318	0.537	0.398	10
k-means++	0.165s	943

**Según la métrica completeness**: El mejor es DBSCAN 0.2-1 sin PCA (los que tienen valor 1.0 no son considerados al tener solo 1 cluster) y el peor es DBSCAN 0.2 sin PCA. 



**Según la métrica  homogeneity**: Los mejores son DBSCAN def sin PCA, DBSCAN 0.7 sin PCA y DBSCAN 0.2 sin PCA y los peores fueron DBSCAN def con PCA, DBSCAN 0.7 con PCA, DBSCAN def-1 con PCA y DBSCAN 0.7-1 con PCA.



**Según la métrica  vmeasure**: El mejor es DBSCAN 0.7 sin PCA y los peores fueron DBSCAN def con PCA, DBSCAN 0.7 con PCA, DBSCAN def 0.7-1 con PCA.





**Según la métrica silhouette**: El mejor fue DBSCAN 0.7 sin PCA y el peor fue DBSCAN 0.2-1 con PCA.

**b)** Indique en cuáles casos no fue posible calcular las métricas de DBSCAN. Analice el número
de clusters obtenidos por DBSCAN y su efecto sobre las métricas. Analice el efecto de
considerar o no los outliers de DBSCAN como un cluster extra y su efecto sobre las
métricas.

**Cuando DBSCAN entregaba solo 1 cluster no era posible calcular "Silhouette" y en las demás métricas se obtienen valores nulos. DBSCAN al aumentar sus clusters se observa una tendencia a aumentar el valor de las métricas.**

**El efecto que tiene considerar los outliers como un cluster adicional es que si es posible calcular "Silhouette", ya que hay por lo menos 2 clusters, el asociado a -1 y el otro que arma de por si el algoritmo. Finalmente al considerar los outliers como cluster el desempeño de las métricas disminuye, lo cual tiene sentido, ya que los outliers son un grupo disperso por todo el espacio.**

**c)** Analice el efecto de usar PCA sobre las métricas obtenidas.

**El primer punto a destacar es que PCA reduce los tiempos de entrenamiento debido al menor volumen de procesamiento de datos. En cuanto a las métricas, PCA tiende a disminuir la calidad de las metricas de la agrupaciones, esto se asocia a que las variables contienen información que se pierde, de modo que la calidad de agrupamiento disminuye. Sin embargo, cabe destacar que hay algunas excepciones en donde las metricas mejoran (ver caso k-means)**

# **Conclusiones:**

**En esta tarea se aprendio a programar algoritmos de clustering y a evaluar los desempeños según distintas métricas. También se comprendio el efecto del PCA en la calidad de los clustering y su tiempo de entrenamiento.**

**En cuanto a objetivos y resultados, se puede decir que se cumplio a cabalidad con todos los requerimientos de la tarea y los resultados obtenidos concuerdan con la teoría, ya que PCA disminuye tiempos de entrenamiento, pero reduce desempeño y en general el mejor algoritmo fue DBSCAN, el cual a su vez tuvo mayor tiempo de entrenamiento al ser un algoritmo de densidad.**

**En cuanto a dificultades, la mayor dificultad fue entender el código base para agregarle nuevo código para implementar otras funcionalidades.**

**Finalmente la mejor solución fue DBSCAN 0.7 sin PCA.**

**Para mejorar el desempeño de los algoritmos, se recomienda hacer un estudio más detallado de los hiperparámetros, como lo es la cantidad de veces que se resuelve k-means para obtener el agrupamiento con mejor desempeño, de modo de encontrar un equilibrio mayor entre tiempo de entrenamiento y desempeño, priorizando en este caso desempeño ya que los tiempos son bajos.**

In [ ]:
!pwd

/content


In [ ]:
#%cd /content/drive/MyDrive/Colab Notebooks 

In [ ]:
#Install:
#!sudo apt-get install texlive-xetex texlive-fonts-recommended texlive-generic-recommended

In [ ]:
#A PDF:
#!jupyter nbconvert --to pdf tarea5ic.ipynb